In [5]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

# Function to mark attendance
def mark_attendance(student_id):
    with open("attend.csv", "a") as file:
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        file.write(f"{student_id},{dt_string}\n")

# Load known faces
known_faces_encodings = []
known_faces_names = []
students_folder = "stud1"  # Folder containing student images
for filename in os.listdir(students_folder):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        student_name = os.path.splitext(filename)[0]
        student_image = face_recognition.load_image_file(os.path.join(students_folder, filename))
        student_encoding = face_recognition.face_encodings(student_image)[0]
        known_faces_encodings.append(student_encoding)
        known_faces_names.append(student_name)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize dictionary to keep track of logged students
logged_students = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Find faces in the frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Loop through each face found in the frame
    for face_encoding, face_location in zip(face_encodings, face_locations):
        # Compare face encoding with known faces
        matches = face_recognition.compare_faces(known_faces_encodings, face_encoding)
        name = "Unknown"
        
        # Check if any known face matches
        if True in matches:
            first_match_index = matches.index(True)
            name = known_faces_names[first_match_index]

            # Check if student has already been logged
            if name not in logged_students:
                mark_attendance(name)
                logged_students[name] = True
        
        # Draw rectangle around the face
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the resulting frame
    cv2.imshow('Attendance System', frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close all windows
cap.release()
cv2.destroyAllWindows()
